In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld

URM_all, ICM_genre_all, ICM_subgenre_all, ICM_channel_all, ICM_event_all = ld.getCOOs()
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13633 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [4]:
from skopt.space import Real, Integer, Categorical

hyperparameters_range_dictionary = {
    "topK": Integer(50, 1000),
    "shrink": Integer(0, 100),
    "similarity": Categorical(["cosine"]),
    "normalize": Categorical([True, False]),
}

In [5]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt

recommender_class = UserKNNCFRecommender

hyperparameterSearch = SearchBayesianSkopt(recommender_class,
                                         evaluator_validation=evaluator_validation,
                                         evaluator_test=evaluator_test)

In [6]:
from HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
  
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train],     # For a CBF model simply put [URM_train, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [7]:
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],     # For a CBF model simply put [URM_train_validation, ICM_train]
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

In [8]:
import os

output_folder_path = "../result_experiments/1_Collaborative"

# If directory does not exist, create
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)
    
n_cases = 50  # using 10 as an example
n_random_starts = int(n_cases*0.3)
metric_to_optimize = "MAP"   
cutoff_to_optimize = 10

In [9]:
hyperparameterSearch.search(recommender_input_args,
                       recommender_input_args_last_test = recommender_input_args_last_test,
                       hyperparameter_search_space = hyperparameters_range_dictionary,
                       n_cases = n_cases,
                       n_random_starts = n_random_starts,
                       save_model = "last",
                       output_folder_path = output_folder_path, # Where to save the results
                       output_file_name_root = recommender_class.RECOMMENDER_NAME, # How to call the files
                       metric_to_optimize = metric_to_optimize,
                       cutoff_to_optimize = cutoff_to_optimize,
                      )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 651, 'shrink': 43, 'similarity': 'cosine', 'normalize': True}
UserKNNCFRecommender: URM Detected 4 ( 0.0%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13650 (100.0%), 631.98 column/sec. Elapsed time 21.60 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 16.11 sec. Users per second: 846
SearchBayesianSkopt: New best config found. Config 0: {'topK': 651, 'shrink': 43, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2292305, PRECISION_RECALL_MIN_DEN: 0.2304923, RECALL: 0.0500024, MAP: 0.1126568, MAP_MIN_DEN: 0.1131720, MRR: 0.4558131, NDCG: 0.2374136, F1: 0.0820968, HIT_RATE: 0.8930536, ARHR_ALL_HITS: 0.7134501, NOVELTY: 0.0054800, AVERAGE_POPULARITY: 0.6124343, DIVERSITY_MEAN_INTER_LIST: 0.8285628, DIVERSITY_HERFINDAHL: 0.9828502, COVERAGE_ITEM: 0.1792458, COVERAGE_ITEM_CORRECT: 0.0384850, COVERAGE_

Similarity column 13650 (100.0%), 688.65 column/sec. Elapsed time 19.82 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 15.74 sec. Users per second: 866
SearchBayesianSkopt: Config 5 is suboptimal. Config: {'topK': 801, 'shrink': 4, 'similarity': 'cosine', 'normalize': False} - results: PRECISION: 0.2023619, PRECISION_RECALL_MIN_DEN: 0.2028375, RECALL: 0.0382373, MAP: 0.0981050, MAP_MIN_DEN: 0.0982825, MRR: 0.4037665, NDCG: 0.2085828, F1: 0.0643209, HIT_RATE: 0.8052520, ARHR_ALL_HITS: 0.6264123, NOVELTY: 0.0060373, AVERAGE_POPULARITY: 0.4636716, DIVERSITY_MEAN_INTER_LIST: 0.9180103, DIVERSITY_HERFINDAHL: 0.9917943, COVERAGE_ITEM: 0.4099895, COVERAGE_ITEM_CORRECT: 0.0926408, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.8042491, DIVERSITY_GINI: 0.0500009, SHANNON_ENTROPY: 8.9118852, RATIO_DIVERSITY_HERFINDAHL: 0.9921769, RATIO_DIVERSITY_GINI: 0.2017178, RATIO_SHANNON_ENTROPY: 0.7192732, RATIO_AVERAGE_POPULARITY: 2.2828407, RATIO_NOVELTY: 0.0355067, 

Iteration No: 6 ended. Evalua

UserKNNCFRecommender: URM Detected 4 ( 0.0%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13650 (100.0%), 745.60 column/sec. Elapsed time 18.31 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 8.50 sec. Users per second: 1604
SearchBayesianSkopt: Config 11 is suboptimal. Config: {'topK': 87, 'shrink': 28, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2179051, PRECISION_RECALL_MIN_DEN: 0.2189363, RECALL: 0.0461921, MAP: 0.1047474, MAP_MIN_DEN: 0.1051158, MRR: 0.4294066, NDCG: 0.2236333, F1: 0.0762256, HIT_RATE: 0.8626861, ARHR_ALL_HITS: 0.6674087, NOVELTY: 0.0057782, AVERAGE_POPULARITY: 0.5006268, DIVERSITY_MEAN_INTER_LIST: 0.9202751, DIVERSITY_HERFINDAHL: 0.9920208, COVERAGE_ITEM: 0.3450357, COVERAGE_ITEM_CORRECT: 0.0941359, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.8616117, DIVERSITY_GINI: 0.0354923, SHANNON_ENTROPY: 8.6718004, RATIO_DIVERSITY_HERFINDAHL: 0.9924035, RATIO_DIVERSITY_GINI: 0.14

EvaluatorHoldout: Processed 13633 (100.0%) in 20.14 sec. Users per second: 677
SearchBayesianSkopt: Config 16 is suboptimal. Config: {'topK': 998, 'shrink': 96, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2275288, PRECISION_RECALL_MIN_DEN: 0.2285917, RECALL: 0.0488665, MAP: 0.1116861, MAP_MIN_DEN: 0.1120768, MRR: 0.4513291, NDCG: 0.2352363, F1: 0.0804539, HIT_RATE: 0.8843982, ARHR_ALL_HITS: 0.7066037, NOVELTY: 0.0055083, AVERAGE_POPULARITY: 0.6083214, DIVERSITY_MEAN_INTER_LIST: 0.8273207, DIVERSITY_HERFINDAHL: 0.9827260, COVERAGE_ITEM: 0.2104768, COVERAGE_ITEM_CORRECT: 0.0418074, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.8832967, DIVERSITY_GINI: 0.0126627, SHANNON_ENTROPY: 7.1336210, RATIO_DIVERSITY_HERFINDAHL: 0.9831052, RATIO_DIVERSITY_GINI: 0.0510849, RATIO_SHANNON_ENTROPY: 0.5757505, RATIO_AVERAGE_POPULARITY: 2.9950093, RATIO_NOVELTY: 0.0323960, 

Iteration No: 17 ended. Search finished for the next optimal point.
Time taken: 46.2828
Function value o

EvaluatorHoldout: Processed 13644 (100.0%) in 19.03 sec. Users per second: 717
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 835, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results:
CUTOFF: 10 - PRECISION: 0.2866168, PRECISION_RECALL_MIN_DEN: 0.2882204, RECALL: 0.0521952, MAP: 0.1523897, MAP_MIN_DEN: 0.1531737, MRR: 0.5292125, NDCG: 0.2968008, F1: 0.0883087, HIT_RATE: 0.9424656, ARHR_ALL_HITS: 0.8909511, NOVELTY: 0.0053489, AVERAGE_POPULARITY: 0.6553159, DIVERSITY_MEAN_INTER_LIST: 0.7869412, DIVERSITY_HERFINDAHL: 0.9786884, COVERAGE_ITEM: 0.0674456, COVERAGE_ITEM_CORRECT: 0.0278531, COVERAGE_USER: 0.9995604, COVERAGE_USER_CORRECT: 0.9420513, DIVERSITY_GINI: 0.0043423, SHANNON_ENTROPY: 6.4327839, RATIO_DIVERSITY_HERFINDAHL: 0.9790659, RATIO_DIVERSITY_GINI: 0.0175182, RATIO_SHANNON_ENTROPY: 0.5191863, RATIO_AVERAGE_POPULARITY: 3.2263821, RATIO_NOVELTY: 0.0314580, 


Iteration No: 22 ended. Search finished for the next optimal point.
Time taken

...

Similarity column 13650 (100.0%), 757.41 column/sec. Elapsed time 18.02 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 7.23 sec. Users per second: 1885
SearchBayesianSkopt: Config 31 is suboptimal. Config: {'topK': 81, 'shrink': 40, 'similarity': 'cosine', 'normalize': False} - results: PRECISION: 0.1530844, PRECISION_RECALL_MIN_DEN: 0.1532486, RECALL: 0.0256537, MAP: 0.0721506, MAP_MIN_DEN: 0.0722200, MRR: 0.3041616, NDCG: 0.1563175, F1: 0.0439434, HIT_RATE: 0.6365437, ARHR_ALL_HITS: 0.4660791, NOVELTY: 0.0069388, AVERAGE_POPULARITY: 0.2702998, DIVERSITY_MEAN_INTER_LIST: 0.9732667, DIVERSITY_HERFINDAHL: 0.9973195, COVERAGE_ITEM: 0.5532975, COVERAGE_ITEM_CORRECT: 0.1709397, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.6357509, DIVERSITY_GINI: 0.0999269, SHANNON_ENTROPY: 10.3521478, RATIO_DIVERSITY_HERFINDAHL: 0.9977043, RATIO_DIVERSITY_GINI: 0.4031331, RATIO_SHANNON_ENTROPY: 0.8355160, RATIO_AVERAGE_POPULARITY: 1.3307942, RATIO_NOVELTY: 0.0408088, 

Iteration No: 32 ended. Sea

Similarity column 13650 (100.0%), 683.13 column/sec. Elapsed time 19.98 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 17.05 sec. Users per second: 800
SearchBayesianSkopt: Config 37 is suboptimal. Config: {'topK': 659, 'shrink': 0, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2319225, PRECISION_RECALL_MIN_DEN: 0.2343396, RECALL: 0.0531953, MAP: 0.1143989, MAP_MIN_DEN: 0.1154241, MRR: 0.4655148, NDCG: 0.2416090, F1: 0.0865410, HIT_RATE: 0.9070637, ARHR_ALL_HITS: 0.7260220, NOVELTY: 0.0053701, AVERAGE_POPULARITY: 0.6451785, DIVERSITY_MEAN_INTER_LIST: 0.7997445, DIVERSITY_HERFINDAHL: 0.9799686, COVERAGE_ITEM: 0.0781328, COVERAGE_ITEM_CORRECT: 0.0294036, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.9059341, DIVERSITY_GINI: 0.0049854, SHANNON_ENTROPY: 6.5812272, RATIO_DIVERSITY_HERFINDAHL: 0.9803467, RATIO_DIVERSITY_GINI: 0.0201125, RATIO_SHANNON_ENTROPY: 0.5311671, RATIO_AVERAGE_POPULARITY: 3.1764716, RATIO_NOVELTY: 0.0315831, 

Iteration No: 38 ended. Searc

SearchBayesianSkopt: Config 42 is suboptimal. Config: {'topK': 51, 'shrink': 1, 'similarity': 'cosine', 'normalize': True} - results: PRECISION: 0.2145529, PRECISION_RECALL_MIN_DEN: 0.2162942, RECALL: 0.0478831, MAP: 0.1028117, MAP_MIN_DEN: 0.1035461, MRR: 0.4285800, NDCG: 0.2213134, F1: 0.0782931, HIT_RATE: 0.8662803, ARHR_ALL_HITS: 0.6606117, NOVELTY: 0.0057855, AVERAGE_POPULARITY: 0.4832743, DIVERSITY_MEAN_INTER_LIST: 0.9339443, DIVERSITY_HERFINDAHL: 0.9933876, COVERAGE_ITEM: 0.3525666, COVERAGE_ITEM_CORRECT: 0.1118556, COVERAGE_USER: 0.9987546, COVERAGE_USER_CORRECT: 0.8652015, DIVERSITY_GINI: 0.0391424, SHANNON_ENTROPY: 8.9182707, RATIO_DIVERSITY_HERFINDAHL: 0.9937708, RATIO_DIVERSITY_GINI: 0.1579115, RATIO_SHANNON_ENTROPY: 0.7197886, RATIO_AVERAGE_POPULARITY: 2.3793525, RATIO_NOVELTY: 0.0340262, 

Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 27.1861
Function value obtained: -0.1028
Current minimum: -0.1145
Iteration No: 44 started. Searching for

C:\Users\marti\miniconda3\envs\RecSys\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 46 ended. Search finished for the next optimal point.
Time taken: 0.6101
Function value obtained: -0.1144
Current minimum: -0.1145
Iteration No: 47 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 1000, 'shrink': 53, 'similarity': 'cosine', 'normalize': False}
UserKNNCFRecommender: URM Detected 4 ( 0.0%) items with no interactions.
Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 13650 (100.0%), 652.87 column/sec. Elapsed time 20.91 sec
EvaluatorHoldout: Processed 13633 (100.0%) in 18.12 sec. Users per second: 752
SearchBayesianSkopt: Config 46 is suboptimal. Config: {'topK': 1000, 'shrink': 53, 'similarity': 'cosine', 'normalize': False} - results: PRECISION: 0.2067483, PRECISION_RECALL_MIN_DEN: 0.2072893, RECALL: 0.0396612, MAP: 0.1003466, MAP_MIN_DEN: 0.1005431, MRR: 0.4115832, NDCG: 0.2130274, F1: 0.0665550, HIT_RATE: 0.8181618, ARHR_ALL_HITS: 0.6394355, NOVELTY: 0.0059488, AVERAGE_POPULARITY: 

In [13]:
from Recommenders.DataIO import DataIO

data_loader = DataIO(folder_path = output_folder_path)
search_metadata = data_loader.load_data(recommender_class.RECOMMENDER_NAME + "_metadata.zip")

search_metadata.keys()

dict_keys(['algorithm_name_recommender', 'algorithm_name_search', 'cutoff_to_optimize', 'exception_list', 'hyperparameters_best', 'hyperparameters_best_index', 'hyperparameters_df', 'metric_to_optimize', 'result_on_last', 'result_on_test_best', 'result_on_test_df', 'result_on_validation_best', 'result_on_validation_df', 'time_df', 'time_on_last_df', 'time_on_test_avg', 'time_on_test_total', 'time_on_train_avg', 'time_on_train_total', 'time_on_validation_avg', 'time_on_validation_total'])

In [16]:
hyperparameters_df = search_metadata["hyperparameters_best"]
hyperparameters_df

{'topK': 324, 'shrink': 0, 'similarity': 'cosine', 'normalize': True}

In [15]:
result_on_validation_df = search_metadata["result_on_validation_df"]
result_on_validation_df

,,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_ITEM_CORRECT,COVERAGE_USER,COVERAGE_USER_CORRECT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
,cutoff,,,,,,,,,,,,,,,,,,,,,
0,10,0.216702,0.218582,0.04796,0.10627,0.107255,0.451879,0.227804,0.078539,0.884207,0.690058,...,0.016446,0.999634,0.883883,0.002433,5.731718,0.972642,0.009829,0.462668,3.488735,0.030756
1,10,0.198051,0.199712,0.042484,0.097737,0.098644,0.436063,0.211581,0.069961,0.851228,0.649777,...,0.003267,0.999634,0.850916,0.00121,4.649879,0.955118,0.004888,0.375341,3.776113,0.030215
2,10,0.19528,0.19696,0.041702,0.096592,0.097548,0.433894,0.209346,0.068727,0.845071,0.644415,...,0.002436,0.999634,0.844762,0.001133,4.555582,0.953094,0.004576,0.367729,3.793276,0.030177
3,10,0.194899,0.196596,0.041582,0.09652,0.097471,0.434693,0.209191,0.06854,0.844119,0.64473,...,0.002381,0.999634,0.84381,0.001123,4.544407,0.952807,0.004536,0.366828,3.795377,0.030173
4,10,0.211513,0.213258,0.046039,0.104431,0.105312,0.448648,0.223676,0.075619,0.873067,0.681465,...,0.010189,0.999634,0.872747,0.001676,5.23377,0.965609,0.006771,0.422473,3.640208,0.030463
5,10,0.213983,0.2158,0.047066,0.104902,0.105802,0.448261,0.225132,0.07716,0.880322,0.682769,...,0.012072,0.999634,0.88,0.00208,5.534669,0.970676,0.008401,0.446762,3.573563,0.030552
6,10,0.195075,0.196788,0.041639,0.096547,0.09751,0.4344,0.209266,0.068629,0.843313,0.64457,...,0.002381,0.999634,0.843004,0.001126,4.547455,0.952883,0.004547,0.367074,3.794919,0.030174
7,10,0.209036,0.210734,0.045319,0.099735,0.100452,0.425312,0.216558,0.074489,0.864492,0.64878,...,0.016834,0.999634,0.864176,0.003391,6.265358,0.980976,0.013698,0.505744,3.355526,0.030893
8,10,0.195346,0.197013,0.0417,0.096606,0.097541,0.434797,0.209464,0.068729,0.845071,0.64515,...,0.002492,0.999634,0.844762,0.001134,4.557139,0.953081,0.004581,0.367855,3.79281,0.030179
